In [1]:
#!/usr/bin/env python
# coding: utf-8
from tqdm import tqdm
import random
import math
import pandas as pd
import pickle
import numpy as np
import operator
from datetime import datetime, timedelta

In [2]:
def expand(cur, need_to_visit, path):
    remain_path = [x for x in need_to_visit if x not in path]
    # remain_path = [x for x in remain_path if x != 'S']
    for child in remain_path:
        child_node = Node(child)
        cur.children.append(child_node)

def backward(path, q):
    cur = n_start
    cur.Q += q
    cur.n += 1
    for cur_state in path[1:]:
        cur = [x for x in cur.children if x.state == cur_state][0]
        cur.Q += q
        cur.n += 1

def ucb_score(Q, n, N, Cp=1):
    """ UCB1 calculation. """
    if n == 0:
        return math.inf
    return (Q / n) + Cp * math.sqrt(math.log(N) / n)

class Node():
    def __init__(self, state, parent=None):
        self.n = 0
        self.Q = 0.0
        self.state = state
        self.children = []
        self.parent = parent

    def expaned(self, child_state):
        for child in child_state:
            child_node = Node(child)
            self.children.append(child_node)

def rollout(need_to_visit, path):
    remain_path = [x for x in need_to_visit if x not in path]
    # remain_path = [x for x in remain_path if x != 'S']
    randomlist = random.sample(range(0, len(remain_path)), len(remain_path))
    random_path = path.copy()
    for idx in randomlist:
        random_path.append(remain_path[idx])
    #     time_ = 0
    #     for state in random_path[1:]:
    #         time_ += random.random()
    time_ = total_time_cal(random_path[1:], cur_time)
    return 360 - time_, random_path

used_features = ['最高温',
                 '最低温',
                 '天气',
                 '风力风向',
                 '空气质量指数',
                 '时间_day',
                 '时间_hour',
                 '时间_minute',
                 '时间_dayofweek',
                 '时间_is_wknd',
                 '项目_等候时间_lag1',
                 '项目_等候时间_lag2',
                 '项目_等候时间_lag3',
                 '项目_等候时间_lag4',
                 '项目_等候时间_lag5',
                 '项目_等候时间_lag6',
                 '项目_等候时间_lag7',
                 '项目_等候时间_lag8',
                 '项目_时间_dayofweek_等候时间_lag1',
                 'past_1h_mean',
                 'past_1h_median',
                 'past_1h_std',
                 'past_1h_min',
                 'past_1h_max',
                 'past_2h_mean',
                 'past_2h_median',
                 'past_2h_std',
                 'past_2h_min',
                 'past_2h_max',
                 '项目_target_mean',
                 '时间_day_target_mean',
                 '时间_hour_target_mean',
                 '时间_minute_target_mean',
                 '时间_dayofweek_target_mean',
                 '时间_is_wknd_target_mean']

target = '等候时间'

test = pd.read_csv("test.csv")
# load model with pickle to predict
with open('model.pkl', 'rb') as fin:
    pkl_bst = pickle.load(fin)


def clip_label(pred, min_, max_):
    pred = np.clip(pred, min_, max_)
    return pred


def time_cal(item, cur_time):
    cur_feas = get_feas(item, cur_time)
    # print(cur_feas)
    y_pred = pkl_bst.predict(cur_feas[used_features])
    y_pred = np.expm1(y_pred)
    y_pred = clip_label(y_pred, 5, 120)
    cur_time = y_pred[0]
    return round(cur_time, 0)


def get_feas(item, cur_time):
    minutes = int(cur_time[14:16])
    if minutes in range(0, 15):
        cur_time = cur_time.split(' ')[0] + ' ' + cur_time.split(' ')[1].replace(cur_time[14:16], '00')
    elif minutes in range(15, 30):
        cur_time = cur_time.split(' ')[0] + ' ' + cur_time.split(' ')[1].replace(cur_time[14:16], '15')
    elif minutes in range(30, 45):
        cur_time = cur_time.split(' ')[0] + ' ' + cur_time.split(' ')[1].replace(cur_time[14:16], '30')
    elif minutes in range(45, 60):
        cur_time = cur_time.split(' ')[0] + ' ' + cur_time.split(' ')[1].replace(cur_time[14:16], '45')
    feas = test.loc[(test['时间'] == cur_time) & (test['项目'] == item)]
    if feas.shape[0] == 0:
        feas = test.loc[(test['项目'] == item)].sample(1)
    # print(cur_time, item)
    #     print(feas)
    return feas


def add_time(cur_time, item_time):
    
    delta_time = timedelta(minutes=item_time)
    cur_time = str(datetime.strptime(cur_time, '%Y-%m-%d %H:%M:%S') + delta_time)
    return cur_time
    
def total_time_cal(path, cur_time):
    time_ = 0
    for item in path:
        item_time = time_cal(item, cur_time)
        #         print(item, item_time)
        time_ += item_time
        cur_time = add_time(cur_time, item_time)
    #         print(cur_time)
    
    return time_

In [3]:
import time
k = 0
while True:
    idx = k % 3
    
    if idx == 0:
        # case1
        need_to_visit = ['功夫熊猫', '变形金刚', '超萌漩漩涡', '哈利波特与禁忌之旅', '飞越侏罗纪' ,'灯影传奇', '奇遇迅猛龙']
        cur_time = "2021-10-19 09:06:00"
    elif idx == 1:
        # case2
        need_to_visit = ['功夫熊猫', '飞越侏罗纪' ,'灯影传奇','奇遇迅猛龙']
        cur_time = "2021-10-19 11:19:00"
    else:
        # case3
        need_to_visit = ['超萌漩漩涡', '哈利波特与禁忌之旅', '奇遇迅猛龙' ,'功夫熊猫']
        cur_time = "2021-10-19 12:06:00"
    print("need_to_visit: ", need_to_visit)
    print("cur_time:", cur_time)
    
    
    # ## 初始化
    n_start = Node("S")
    n_start.expaned(need_to_visit)

    # ## 迭代
    for _ in tqdm(range(30)):

        N = n_start.n
        path = []

        cur = n_start
        path.append(cur.state)
        while cur.children != []:
            ucb_of_children = []
            for child in cur.children:
                Q = child.Q
                n = child.n
                ucb_ = ucb_score(Q, n, N)
                ucb_of_children.append(ucb_)
            max_ucb_idx_ = ucb_of_children.index(max(ucb_of_children))
            cur = cur.children[max_ucb_idx_]
            path.append(cur.state)

        if cur.n == 0:
            q, random_path = rollout(need_to_visit, path)
            backward(path, q)
        else:
            expand(cur, need_to_visit, path)
            if len(cur.children) == 0:
                pass
            else:
                cur = cur.children[0]
                path.append(cur.state)
            q, random_path = rollout(need_to_visit, path)
            backward(path, q)
            
    # 景点推荐
    score_of_children = {}
    for child in n_start.children:
        Q = child.Q
        n = child.n
        score_ = Q / n
        score_of_children[child.state] = score_
    print('得分:', score_of_children)
    recommend = max(score_of_children.items(), key=operator.itemgetter(1))[0]

    print('推荐景点: ', recommend)

    res = pd.DataFrame()
    res['recommend'] = [recommend]
    res.to_csv(f'./sub/recommend.txt', index = False, header = False)
    
    time.sleep(10)
    print("#" * 30)
    print()
        
    k += 1

need_to_visit:  ['功夫熊猫', '变形金刚', '超萌漩漩涡', '哈利波特与禁忌之旅', '飞越侏罗纪', '灯影传奇', '奇遇迅猛龙']
cur_time: 2021-10-19 09:06:00


100%|██████████| 30/30 [00:03<00:00,  8.18it/s]


得分: {'功夫熊猫': 193.0, '变形金刚': 163.0, '超萌漩漩涡': 148.0, '哈利波特与禁忌之旅': 128.0, '飞越侏罗纪': 208.56521739130434, '灯影传奇': 158.0, '奇遇迅猛龙': 185.0}
推荐景点:  飞越侏罗纪
##############################

need_to_visit:  ['功夫熊猫', '飞越侏罗纪', '灯影传奇', '奇遇迅猛龙']
cur_time: 2021-10-19 11:19:00


100%|██████████| 30/30 [00:02<00:00, 14.52it/s]


得分: {'功夫熊猫': 231.0, '飞越侏罗纪': 220.0, '灯影传奇': 237.37037037037038, '奇遇迅猛龙': 233.0}
推荐景点:  灯影传奇
##############################

need_to_visit:  ['超萌漩漩涡', '哈利波特与禁忌之旅', '奇遇迅猛龙', '功夫熊猫']
cur_time: 2021-10-19 12:06:00


100%|██████████| 30/30 [00:02<00:00, 14.11it/s]


得分: {'超萌漩漩涡': 263.0, '哈利波特与禁忌之旅': 265.0, '奇遇迅猛龙': 262.0, '功夫熊猫': 267.74074074074076}
推荐景点:  功夫熊猫
##############################

need_to_visit:  ['功夫熊猫', '变形金刚', '超萌漩漩涡', '哈利波特与禁忌之旅', '飞越侏罗纪', '灯影传奇', '奇遇迅猛龙']
cur_time: 2021-10-19 09:06:00


100%|██████████| 30/30 [00:03<00:00,  8.42it/s]


得分: {'功夫熊猫': 185.0, '变形金刚': 170.0, '超萌漩漩涡': 149.0, '哈利波特与禁忌之旅': 155.0, '飞越侏罗纪': 195.47826086956522, '灯影传奇': 166.0, '奇遇迅猛龙': 141.0}
推荐景点:  飞越侏罗纪


KeyboardInterrupt: 